# LSTM

In [ ]:
from src.rnn.architecture import RnnArch
from src.rnn.kfold import RnnFoldCrossValidation
from src.rnn.model import RnnConstructorArgs, RnnModelInitializeArgs, RnnIntMultiRunTrainArgs,\
    RnnIntTrainArgs

from src.common.model import DEFAULT_DATASET, DEFAULT_FULL_DATASET


## Model Selection

In [ ]:
# Control
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="control-model",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH0,
            input_width=5,
            spacing=1
        ),
        dataset=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=5,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# Dnn mimic
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="dnn-mimic-kf",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.DNNMIMIC,
            input_width=1,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=5,
    train_args=RnnIntTrainArgs(
        epochs=10,
        balanced=False,
        augmented=False
    )
))

In [ ]:
# RNN arch 1, small width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-small-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=5,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 1, medium width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-medium-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=15,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 1, large width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch1-large-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH1,
            input_width=30,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 2, medium width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch2-medium-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH2,
            input_width=15,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(train_run_args=RnnIntMultiRunTrainArgs(
    runs=10,
    train_args=RnnIntTrainArgs(
        epochs=10
    )
))

In [ ]:
# RNN arch 4, medium width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch4-medium-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH4,
            input_width=15,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, train_args=RnnIntTrainArgs(epochs=10)
    )
)

In [ ]:
# RNN arch 5, medium width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch5-medium-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH5,
            input_width=15,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, 
        train_args=RnnIntTrainArgs(
            epochs=10
        )
    )
)

In [ ]:
# RNN arch 5, medium width, dense, balanced
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch5-medium-dense-balanced",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH5,
            input_width=15,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, 
        train_args=RnnIntTrainArgs(
            epochs=10,
            balanced=True
        )
    )
)

In [ ]:
# RNN arch 5, medium width, dense, augmented
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch5-medium-dense-augmented",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH5,
            input_width=15,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=4, 
        train_args=RnnIntTrainArgs(
            epochs=10
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=1, 
        train_args=RnnIntTrainArgs(
            epochs=10,
            augmented=True
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=4, 
        train_args=RnnIntTrainArgs(
            epochs=10
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=1, 
        train_args=RnnIntTrainArgs(
            epochs=10,
            augmented=True
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=4, 
        train_args=RnnIntTrainArgs(
            epochs=10
        )
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=1, 
        train_args=RnnIntTrainArgs(
            epochs=10,
            augmented=True
        )
    )
)

In [ ]:
# RNN arch 6, medium width, dense
rnn_folds = RnnFoldCrossValidation(
    model_args=RnnConstructorArgs(
        name="arch6-medium-dense",
        model_initialize_args=RnnModelInitializeArgs(
            model_arch=RnnArch.ARCH6,
            input_width=15,
            spacing=1
        ),
        dataset_name=DEFAULT_FULL_DATASET
    )
)
rnn_folds.train_folds(
    train_run_args=RnnIntMultiRunTrainArgs(
        runs=10, 
        train_args=RnnIntTrainArgs(
            epochs=10
        )
    )
)